In [1]:
import numpy as np
import pandas as pd

In [2]:
import pathlib
import re
import os

# Read IMDB Dataset

In [3]:
df_ls = []



basepath="../data/aclImdb/"
set_name = "train"

tags = ["pos","neg"]


for tag in tags:
    id_ls = []
    score_ls = []
    text_ls = []
    label_ls = []

    url_ls = []
    tconst_ls = []
    
    path = pathlib.Path(basepath, set_name, tag)
    url_file = pathlib.Path(basepath, set_name, f"urls_{tag}.txt")
    
    with open(url_file, "r") as f:
        _allurls = f.readlines()
        url_ls.append([x.rstrip() for x in _allurls])
        tconst_ls.append([x.rstrip().split("/")[-2] for x in _allurls])
    
    for ifile in path.glob("*"):
        with open(ifile, "r") as f:
            txt = f.readlines()

            text_ls.append(" ".join(txt))





        _id, _score = ifile.with_suffix('').name.split("_")

        id_ls.append(int(_id))
        score_ls.append(_score)
        label_ls.append(tag)

    df_tmp = pd.DataFrame({"id":id_ls,
                         "text":text_ls,
                         "label":label_ls,
                         "score":score_ls,
                        })
    
    df_tmp = df_tmp.set_index('id')

    df_url = pd.DataFrame({"url":url_ls[0], "tconst":tconst_ls[0]})

    df_url.index.name = 'id'

    df_tmp = df_tmp.merge(df_url, on='id')



    assert df_tmp.index.nunique() == len(df_tmp)

    df_tmp['id_w_tag'] = df_tmp.index

    df_tmp['id_w_tag'] = df_tmp['id_w_tag'].apply(lambda x: tag + "_" + str(x))
    df_tmp = df_tmp.reset_index()
    
    df_tmp['set'] = set_name
    
    
    df_ls.append(df_tmp)

In [4]:
df = pd.concat(df_ls, ignore_index=True)

In [5]:
df['id_w_tag'].nunique() == len(df)

True

In [6]:
df

,id,text,label,score,url,tconst,id_w_tag,set
0,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train
1,11274,One of the best Tarzan films is also one of it...,pos,9,http://www.imdb.com/title/tt0025862/usercomments,tt0025862,pos_11274,train
2,1759,"I'm not quite sure if the term ""serious comedy...",pos,8,http://www.imdb.com/title/tt0405422/usercomments,tt0405422,pos_1759,train
3,2053,First of all i'd just like to say this movie r...,pos,10,http://www.imdb.com/title/tt0102422/usercomments,tt0102422,pos_2053,train
4,7638,Larry Burrows has the distinct feeling he's mi...,pos,7,http://www.imdb.com/title/tt0100201/usercomments,tt0100201,pos_7638,train
...,...,...,...,...,...,...,...,...
24995,7867,"Okay, I've tried and I've tried, but I STILL D...",neg,2,http://www.imdb.com/title/tt0346800/usercomments,tt0346800,neg_7867,train
24996,7113,THE SCREAMING SKULL (1 outta 5 stars) This mov...,neg,2,http://www.imdb.com/title/tt0052169/usercomments,tt0052169,neg_7113,train
24997,3075,Yep.. this is definatly up there with some of ...,neg,1,http://www.imdb.com/title/tt0089280/usercomments,tt0089280,neg_3075,train
24998,10258,The Earth is destined to be no more thanks to ...,neg,4,http://www.imdb.com/title/tt0075990/usercomments,tt0075990,neg_10258,train


In [7]:
df_bak = df.copy()

In [1]:
df_bak

NameError: name 'df_bak' is not defined

# Get Genre

In [8]:
df_title_basics = pd.read_csv("../data/title.basics.tsv", sep="\t")

/home/NETID/xiruod/anaconda3/envs/cognitive_test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
df_title_basics["tconst"].nunique()

9650055

In [10]:
len(df_title_basics)

9650055

# Merge Genre

In [11]:
df = df.merge(df_title_basics, on="tconst", how='inner')

In [12]:
df

,id,text,label,score,url,tconst,id_w_tag,set,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,movie,The Best of Times,The Best of Times,0,1986,\N,104,"Comedy,Drama,Sport"
1,11761,Kurt Russell is at his best as the man who liv...,pos,10,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11761,train,movie,The Best of Times,The Best of Times,0,1986,\N,104,"Comedy,Drama,Sport"
2,11763,I guess that everyone has to make a comeback a...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11763,train,movie,The Best of Times,The Best of Times,0,1986,\N,104,"Comedy,Drama,Sport"
3,11755,"I love this movie, but the music at all the al...",pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11755,train,movie,The Best of Times,The Best of Times,0,1986,\N,104,"Comedy,Drama,Sport"
4,11752,For all of the Has-Beens or Never Was's or for...,pos,10,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11752,train,movie,The Best of Times,The Best of Times,0,1986,\N,104,"Comedy,Drama,Sport"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23791,11113,"I often feel like Scrooge, slamming movies tha...",neg,3,http://www.imdb.com/title/tt0045815/usercomments,tt0045815,neg_11113,train,movie,The Girl Next Door,The Girl Next Door,0,1953,\N,92,"Comedy,Musical"
23792,4237,JUDAAI was a bold film by Raj Kanwar at it's t...,neg,3,http://www.imdb.com/title/tt0119427/usercomments,tt0119427,neg_4237,train,movie,Judaai,Judaai,0,1997,\N,167,"Comedy,Drama,Romance"
23793,10528,I picked this one up because the music was don...,neg,3,http://www.imdb.com/title/tt0094327/usercomments,tt0094327,neg_10528,train,video,The Wind,The Wind,0,1986,\N,92,"Horror,Thriller"
23794,11231,I love low budget independent films and had hi...,neg,3,http://www.imdb.com/title/tt0346899/usercomments,tt0346899,neg_11231,train,video,Dr. Jekyll and Mr. Hyde,Dr. Jekyll and Mr. Hyde,0,2002,\N,109,"Drama,Horror,Sci-Fi"


In [13]:
# df.to_csv("train_w_genre.csv", index=False)



# df = pd.read_csv("train_w_genre.csv")

In [14]:
df['id_w_tag'].nunique() == len(df)

True

In [15]:
genres_ls = df['genres'].unique().tolist()

genres_ls = [x.split(",") for x in genres_ls]

In [16]:
horror_neighbor = set()
tmp = [horror_neighbor.update(x) for x in genres_ls if "Horror" in x]

In [17]:
all_genres = set()
tmp = [all_genres.update(x) for x in genres_ls]

In [18]:
all_genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western',
 '\\N'}

In [19]:
all_genres.difference(horror_neighbor)

{'Biography',
 'Documentary',
 'Family',
 'Film-Noir',
 'Game-Show',
 'News',
 'Reality-TV',
 'Sport',
 'Talk-Show',
 '\\N'}

In [20]:
[x for x in genres_ls if ("Horror" in x) and ("Family" in x)]

[]

In [21]:
df.groupby("label").size()

label
neg    11912
pos    11884
dtype: int64

In [22]:
df[df['genres'].str.contains("Horror")].groupby("label").size()

label
neg    3055
pos    1167
dtype: int64

In [23]:
for i in all_genres.difference(horror_neighbor):
    if i == "\\N":
        continue
    print(i)
    print(df[df['genres'].str.contains(i)].groupby("label").size())

Film-Noir
label
neg     40
pos    204
dtype: int64
News
label
neg    31
pos     3
dtype: int64
Sport
label
neg    141
pos    201
dtype: int64
Reality-TV
label
neg    40
pos    13
dtype: int64
Biography
label
neg    348
pos    689
dtype: int64
Game-Show
label
neg    22
pos     8
dtype: int64
Talk-Show
label
neg    16
pos    14
dtype: int64
Documentary
label
neg    276
pos    497
dtype: int64
Family
label
neg    612
pos    782
dtype: int64


In [24]:
df_horror = df[df['genres'].str.contains("Horror")].reset_index(drop=True)
df_horror['domain'] = 'Horror'

In [31]:
df_nonhorror = df[~df['genres'].str.contains("Horror")].reset_index(drop=True)
df_nonhorror['domain'] = 'nonHorror'

In [25]:
df_family = df[df['genres'].str.contains("Family")].reset_index(drop=True)
df_family['domain'] = 'Family'

In [26]:
df_documentary = df[df['genres'].str.contains("Documentary")].reset_index(drop=True)
df_documentary['domain'] = 'Documentary'

In [27]:
df_mix = pd.concat([df_horror, df_documentary]).reset_index(drop=True)

In [28]:
df_mix.to_csv("../data/aclImdb/horror_documentary.csv", index=False)

In [30]:
df_mix.groupby(["domain",'label']).size()

domain       label
Documentary  neg       276
             pos       497
Horror       neg      3055
             pos      1167
dtype: int64

In [33]:
df_mix = pd.concat([df_horror, df_nonhorror]).reset_index(drop=True)

In [34]:
df_mix.to_csv("../data/aclImdb/horror_nonhorror.csv", index=False)

In [35]:
df_mix.groupby(["domain",'label']).size()

domain     label
Horror     neg       3055
           pos       1167
nonHorror  neg       8857
           pos      10717
dtype: int64

In [32]:
df_mix = pd.concat([df_horror, df_family]).reset_index(drop=True)

In [33]:
df_mix

,id,text,label,score,url,tconst,id_w_tag,set,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,domain
0,8286,Larry Fessenden has been thrashed by most of t...,pos,7,http://www.imdb.com/title/tt0275067/usercomments,tt0275067,pos_8286,train,movie,Wendigo,Wendigo,0,2001,\N,91,"Horror,Mystery,Thriller",Horror
1,8279,This film is more about how children make sens...,pos,10,http://www.imdb.com/title/tt0275067/usercomments,tt0275067,pos_8279,train,movie,Wendigo,Wendigo,0,2001,\N,91,"Horror,Mystery,Thriller",Horror
2,8273,If you want Scream or anything like the big-st...,pos,10,http://www.imdb.com/title/tt0275067/usercomments,tt0275067,pos_8273,train,movie,Wendigo,Wendigo,0,2001,\N,91,"Horror,Mystery,Thriller",Horror
3,8276,"Now this is what I'm talking about. Finally, a...",pos,10,http://www.imdb.com/title/tt0275067/usercomments,tt0275067,pos_8276,train,movie,Wendigo,Wendigo,0,2001,\N,91,"Horror,Mystery,Thriller",Horror
4,8281,"From the first scene, I was really excited. ""I...",pos,7,http://www.imdb.com/title/tt0275067/usercomments,tt0275067,pos_8281,train,movie,Wendigo,Wendigo,0,2001,\N,91,"Horror,Mystery,Thriller",Horror
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5611,4268,I remembered this show from when i was a kid. ...,neg,1,http://www.imdb.com/title/tt0366068/usercomments,tt0366068,neg_4268,train,tvSeries,Photon,Photon,0,1986,\N,\N,"Action,Adventure,Family",Family
5612,4519,"Three Stooges - Have Rocket, Will Travel - 195...",neg,3,http://www.imdb.com/title/tt0052880/usercomments,tt0052880,neg_4519,train,movie,Have Rocket -- Will Travel,Have Rocket -- Will Travel,0,1959,\N,76,"Comedy,Family,Sci-Fi",Family
5613,11260,"This movie should be called ""plan 9 from josep...",neg,4,http://www.imdb.com/title/tt0382958/usercomments,tt0382958,neg_11260,train,video,Saturday's Warrior,Saturday's Warrior,0,1989,\N,115,"Drama,Family,Musical",Family
5614,11474,This relic of a short film starts with a teen ...,neg,1,http://www.imdb.com/title/tt0150255/usercomments,tt0150255,neg_11474,train,tvMovie,Dad... Can I Borrow the Car?,Dad... Can I Borrow the Car?,0,1970,\N,22,"Comedy,Family",Family


In [34]:
df_mix.to_csv("../data/aclImdb/horror_family.csv", index=False)

# Region

In [9]:
df_akas = pd.read_csv("../data/title.akas.tsv", sep="\t")

/home/NETID/xiruod/anaconda3/envs/cognitive_test/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
df_akas.query("titleId == 'tt0044083'")

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
277309,tt0044083,10,El bulevar de las pasiones,MX,\N,imdbDisplay,\N,0
277310,tt0044083,11,Стрип,SUHH,ru,imdbDisplay,\N,0
277311,tt0044083,1,Pasek,PL,\N,imdbDisplay,\N,0
277312,tt0044083,2,Tödliches Pflaster Sunset Strip,XWG,\N,\N,\N,0
277313,tt0044083,3,A vetkőzés,HU,\N,imdbDisplay,\N,0
277314,tt0044083,4,Jazzen går vidare,SE,\N,\N,\N,0
277315,tt0044083,5,Amei e Errei,BR,\N,imdbDisplay,\N,0
277316,tt0044083,6,La donna del gangster,IT,\N,\N,\N,0
277317,tt0044083,7,Jäljet johtavat yökerhoon,FI,\N,\N,\N,0
277318,tt0044083,8,The Strip,US,\N,imdbDisplay,\N,0


In [20]:
df_akas[df_akas['titleId'] == 'tt0090713']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
742484,tt0090713,10,Gençlik Sevdası,TR,tr,imdbDisplay,new title,0
742485,tt0090713,11,Los buenos tiempos,ES,\N,alternative,\N,0
742486,tt0090713,12,The Best of Times,US,\N,imdbDisplay,\N,0
742487,tt0090713,13,"Most kapd el, Jack!",HU,\N,imdbDisplay,\N,0
742488,tt0090713,14,Seni geri laikai,SUHH,lt,imdbDisplay,\N,0
742489,tt0090713,15,Viimeinen ottelu,FI,\N,\N,\N,0
742490,tt0090713,16,Najljepsa vremena,HR,\N,\N,\N,0
742491,tt0090713,17,Rocket Man - Der Beste aller Zeiten,DE,\N,imdbDisplay,\N,0
742492,tt0090713,18,Rocket Man - Der Beste aller Zeiten,XWG,\N,\N,\N,0
742493,tt0090713,19,A Última Chance,BR,\N,\N,\N,0


In [14]:
df_merge = df_bak.merge(df_akas, left_on="tconst", right_on="titleId", how="inner")

In [15]:
df_merge

,id,text,label,score,url,tconst,id_w_tag,set,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,tt0090713,10,Gençlik Sevdası,TR,tr,imdbDisplay,new title,0
1,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,tt0090713,11,Los buenos tiempos,ES,\N,alternative,\N,0
2,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,tt0090713,12,The Best of Times,US,\N,imdbDisplay,\N,0
3,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,tt0090713,13,"Most kapd el, Jack!",HU,\N,imdbDisplay,\N,0
4,11758,Spoiler This movie is about such a concept. Wi...,pos,7,http://www.imdb.com/title/tt0090713/usercomments,tt0090713,pos_11758,train,tt0090713,14,Seni geri laikai,SUHH,lt,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554318,5798,"This movie is a rather odd mix of musical, rom...",neg,4,http://www.imdb.com/title/tt0044083/usercomments,tt0044083,neg_5798,train,tt0044083,5,Amei e Errei,BR,\N,imdbDisplay,\N,0
554319,5798,"This movie is a rather odd mix of musical, rom...",neg,4,http://www.imdb.com/title/tt0044083/usercomments,tt0044083,neg_5798,train,tt0044083,6,La donna del gangster,IT,\N,\N,\N,0
554320,5798,"This movie is a rather odd mix of musical, rom...",neg,4,http://www.imdb.com/title/tt0044083/usercomments,tt0044083,neg_5798,train,tt0044083,7,Jäljet johtavat yökerhoon,FI,\N,\N,\N,0
554321,5798,"This movie is a rather odd mix of musical, rom...",neg,4,http://www.imdb.com/title/tt0044083/usercomments,tt0044083,neg_5798,train,tt0044083,8,The Strip,US,\N,imdbDisplay,\N,0


In [17]:
df_bak.shape

(25000, 8)